# **AML Mistake Recognition - LSTM Baseline Training**

This notebook trains and evaluates the new LSTM baseline for mistake detection in procedural activities.

- Train LSTM baseline from scratch
- Compare with MLP and Transformer baselines
- Evaluate on step and recordings splits
- Generate comparison visualizations (optional)

# **1 Initial Setup**

## 1.1 Install dependencies

In [ ]:
!pip install torcheval wandb -q

## 1.2 Import the project (with LSTM baseline)

In [ ]:
## delete /code if it already exists and you need to reclone the project
!rm -rf code

In [ ]:
# clone the project repository
!git clone --recursive https://github.com/SimoneColu/AML_error_recognition.git code

### 1.2.1 Add LSTM Implementation Files

In [ ]:
# from Google Drive 
from google.colab import drive
drive.mount('/content/drive')

# copy files from Drive to the project
# !cp /content/drive/MyDrive/AML_DAAI_25_26/er_lstm.py code/core/models/
# !cp /content/drive/MyDrive/AML_DAAI_25_26/base.py code/
# !cp /content/drive/MyDrive/AML_DAAI_25_26/constants.py code/

### 1.2.2 Verify LSTM Files Are Present

In [ ]:
# Check that LSTM model file exists
!ls -lh code/core/models/er_lstm.py

# Check that constants have LSTM variants
!grep "LSTM_VARIANT" code/constants.py

# Check that base.py supports LSTM
!grep -A 3 "LSTM_VARIANT" code/base.py | head -10

## 1.3 Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1.4 Set the constants

In [ ]:
DRIVE_PATH = "/content/drive/MyDrive/AML_DAAI_25_26"
LOCAL_CODE_PATH = "/content/code"

## Create necessary directories
!mkdir -p {LOCAL_CODE_PATH}/data/video
!mkdir -p {LOCAL_CODE_PATH}/checkpoints/error_recognition/LSTM/omnivore
!mkdir -p {LOCAL_CODE_PATH}/checkpoints/error_recognition/LSTM/slowfast
!mkdir -p {LOCAL_CODE_PATH}/checkpoints/error_recognition/GRU/omnivore
!mkdir -p {LOCAL_CODE_PATH}/checkpoints/error_recognition/LSTM_Attention/omnivore
!mkdir -p {LOCAL_CODE_PATH}/results
!mkdir -p {LOCAL_CODE_PATH}/plots

# **2 Extract Features**

## 2.1 Extract Omnivore Features

In [ ]:
# Extract Omnivore features (quiet mode)
!unzip -q "{DRIVE_PATH}/data/backbone/omnivore.zip" -d {LOCAL_CODE_PATH}/data/video

## 2.2 Extract SlowFast Features (Optional)

to train on SlowFast features as well

In [ ]:
# !unzip -q "{DRIVE_PATH}/data/backbone/slowfast.zip" -d {LOCAL_CODE_PATH}/data/video

# **3 Train LSTM Baseline**

## 3.1 Train LSTM on Step Split (Omnivore)

This will train the LSTM model on the step split. Expected training time: **1-2 hours**

In [ ]:
%%bash

cd code
python train_er.py \
  --variant LSTM \
  --backbone omnivore \
  --split step \
  --num_epochs 50 \
  --lr 1e-3 \
  --batch_size 1 \
  --weight_decay 1e-3 \
  --seed 42 \
  --ckpt_directory /content/code/checkpoints

## 3.2 Train LSTM on Recordings Split (Omnivore)

This will train on the more challenging recordings split.

In [ ]:
%%bash

cd code
python train_er.py \
  --variant LSTM \
  --backbone omnivore \
  --split recordings \
  --num_epochs 50 \
  --lr 1e-3 \
  --batch_size 1 \
  --weight_decay 1e-3 \
  --seed 42 \
  --ckpt_directory /content/code/checkpoints

## 3.4 Monitor Training Progress

Check the training logs to see how the model is performing.

In [ ]:
# View training statistics
!tail -20 code/stats/error_recognition/LSTM/omnivore/*_training_performance.txt

# **4 Evaluate LSTM Baseline**

## 4.1 Evaluate LSTM - Step Split

In [ ]:
%%bash

cd code
python -m core.evaluate \
  --variant LSTM \
  --backbone omnivore \
  --ckpt checkpoints/error_recognition/LSTM/omnivore/error_recognition_step_omnivore_LSTM_video_best.pt \
  --split step \
  --threshold 0.6

## 4.2 Evaluate LSTM - Recordings Split

In [ ]:
%%bash

cd code
python -m core.evaluate \
  --variant LSTM \
  --backbone omnivore \
  --ckpt checkpoints/error_recognition/LSTM/omnivore/error_recognition_recordings_omnivore_LSTM_video_best.pt \
  --split recordings \
  --threshold 0.4

# **5 Compare with Existing Baselines**

## 5.1 Evaluate MLP Baseline (if checkpoint available)

In [ ]:
# Copy MLP checkpoint from Drive
!cp "{DRIVE_PATH}/data/checkpoint/MLP/error_recognition_MLP_omnivore_step_epoch_43.pt" code/checkpoints/error_recognition_best/MLP/omnivore/

In [ ]:
%%bash

cd code
python -m core.evaluate \
  --variant MLP \
  --backbone omnivore \
  --ckpt checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_step_epoch_43.pt \
  --split step \
  --threshold 0.6

## 5.2 Evaluate Transformer Baseline (if checkpoint available)

In [ ]:
# Copy Transformer checkpoint from Drive
!cp "{DRIVE_PATH}/data/checkpoint/Transformer/error_recognition_Transformer_omnivore_step_epoch_9.pt" code/checkpoints/error_recognition_best/Transformer/omnivore/

In [ ]:
%%bash

cd code
python -m core.evaluate \
  --variant Transformer \
  --backbone omnivore \
  --ckpt checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_step_epoch_9.pt \
  --split step \
  --threshold 0.6